In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
!git pull

Already up to date.


In [19]:
! pip install mlflow transformers datasets "pydantic[dotenv]" av

In [20]:
import mlflow
from mlflow.client import MlflowClient
import pandas as pd
import os
import matplotlib.pyplot as plt
from overfit.env_settings import settings
from overfit.utils.mlflow import get_all_experiments, get_params_from_experiment_name
import numpy as np
import itertools
from tqdm.notebook import tqdm


In [ ]:
assert settings.MLFLOW_TRACKING_URI


In [22]:
client = MlflowClient(tracking_uri=settings.MLFLOW_TRACKING_URI)


In [23]:
ALL_CONFIDENCES = [0.1, 0.2, 0.3, 0.9]
ALL_WEIGHT_DECAYS = [0.1, 0.2, 0.3, 0.4]
ALL_MAX_LRS = [0.1, 0.2, 0.3, 0.4]
ALL_DATSETS = ["4-50", "5-50", "6-75"]
MODELS = ["resnet18", "resnet34", "resnet50", "vit"]


In [24]:
CONFIDENCES = [0.0, 0.1, 0.2, 0.3, 0.9]
WEIGHT_DECAYS = [0.2]
MAX_LRS = [0.4]
MOMENTUMS = [0.1]
DATASETS = ["4-50"]
MODELS = ["vit"]

EXPERIMENT_NAMES = get_all_experiments(
    datasets=DATASETS,
    models=MODELS,
    confidences=CONFIDENCES,
    weight_decays=WEIGHT_DECAYS,
    max_lrs=MAX_LRS,
    momentums=MOMENTUMS,
)
EXPERIMENT_NAMES


['D4-50MvitC0.0WD0.2LR0.4M0.1',
 'D4-50MvitC0.1WD0.2LR0.4M0.1',
 'D4-50MvitC0.2WD0.2LR0.4M0.1',
 'D4-50MvitC0.3WD0.2LR0.4M0.1',
 'D4-50MvitC0.9WD0.2LR0.4M0.1']

In [25]:
experiments = [
    client.get_experiment_by_name(experiment_name)
    for experiment_name in tqdm(EXPERIMENT_NAMES)
]


  0%|          | 0/5 [00:00<?, ?it/s]

In [11]:
experiments


[<Experiment: artifact_location='mlflow-artifacts:/52', creation_time=1669509831923, experiment_id='52', last_update_time=1669509831923, lifecycle_stage='active', name='D4-50MvitC0.0WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/38', creation_time=1669120072652, experiment_id='38', last_update_time=1669120072652, lifecycle_stage='active', name='D4-50MvitC0.2WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/39', creation_time=1669120072692, experiment_id='39', last_update_time=1669120072692, lifecycle_stage='active', name='D4-50MvitC0.3WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/40', creation_time=1669120072714, experiment_id='40', last_update_time=1669120072714, lifecycle_stage='active', n

In [12]:
def get_experiment_error_df(experiment_id: str) -> pd.DataFrame:
    src_errors = []
    tgt_errors = []
    runs = client.search_runs(experiment_ids=[experiment_id])
    for run in tqdm(runs):
        N = int(run.data.params["Frames"])
        filename = run.data.params["Filename"]
        src_acc_top1 = run.data.metrics["Source Accumulated Top-1 Count"]
        tgt_acc_top1 = run.data.metrics["Target Accumulated Top-1 Count"]
        src_error = 100 * (N - src_acc_top1) / N
        tgt_error = 100 * (N - tgt_acc_top1) / N
        assert src_error >= 0, (src_error, src_acc_top1, N, filename)
        assert tgt_error >= 0, (tgt_error, tgt_acc_top1, N, filename)
        src_errors.append(src_error)
        tgt_errors.append(tgt_error)

    df = pd.DataFrame(
        {
            "Source Error": src_errors,
            "Target Error": tgt_errors,
        }
    )
    return df.describe()


In [13]:
dfs = [
    get_experiment_error_df(experiment.experiment_id)
    for experiment in tqdm(experiments)
]


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

  0%|          | 0/976 [00:00<?, ?it/s]

In [14]:
experiment_names = [experiment.name for experiment in experiments]


In [15]:
params_df = pd.DataFrame(
    [
        get_params_from_experiment_name(experiment_name)
        for experiment_name in experiment_names
    ],
    index=experiment_names,
)
params_df


,dataset,model,confidence,weight_decay,max_lr,momentum
D4-50MvitC0.0WD0.2LR0.4M0.1,4-50,vit,0.0,0.2,0.4,0.1
D4-50MvitC0.1WD0.2LR0.4M0.1,4-50,vit,0.1,0.2,0.4,0.1
D4-50MvitC0.2WD0.2LR0.4M0.1,4-50,vit,0.2,0.2,0.4,0.1
D4-50MvitC0.3WD0.2LR0.4M0.1,4-50,vit,0.3,0.2,0.4,0.1
D4-50MvitC0.9WD0.2LR0.4M0.1,4-50,vit,0.9,0.2,0.4,0.1


In [16]:
mean_df = pd.concat([df.loc["mean"] for df in dfs], axis=1).T
mean_df = mean_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
mean_df = mean_df.rename(
    columns={"Source Error": "Source Error Mean", "Target Error": "Target Error Mean"}
)
mean_df_base = mean_df[["Source Error Mean", "Target Error Mean"]]
mean_df


,Source Error Mean,Target Error Mean
D4-50MvitC0.0WD0.2LR0.4M0.1,59.351015,52.256427
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648
D4-50MvitC0.2WD0.2LR0.4M0.1,59.351015,55.143210
D4-50MvitC0.3WD0.2LR0.4M0.1,59.351015,55.883476
D4-50MvitC0.9WD0.2LR0.4M0.1,59.351015,56.938571


In [17]:
mean_df["Error Mean Percentual Change"] = (
    100
    * (mean_df["Target Error Mean"] - mean_df["Source Error Mean"])
    / mean_df["Source Error Mean"]
)
# mean_df["Error Mean Absolute Change"] = mean_df["Target Error Mean"] - mean_df["Source Error Mean"]
mean_df_change = mean_df[["Error Mean Percentual Change"]]
mean_df


,Source Error Mean,Target Error Mean,Error Mean Percentual Change
D4-50MvitC0.0WD0.2LR0.4M0.1,59.351015,52.256427,-11.953609
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124
D4-50MvitC0.2WD0.2LR0.4M0.1,59.351015,55.143210,-7.089694
D4-50MvitC0.3WD0.2LR0.4M0.1,59.351015,55.883476,-5.842426
D4-50MvitC0.9WD0.2LR0.4M0.1,59.351015,56.938571,-4.064706


In [18]:
std_df = pd.concat([df.loc["std"] for df in dfs], axis=1).T
std_df = std_df.set_axis(
    [experiment.name for experiment in experiments], axis=0
).dropna()
std_df = std_df.rename(
    columns={"Source Error": "Source Error Std", "Target Error": "Target Error Std"}
)
std_df_base = std_df[["Source Error Std", "Target Error Std"]]
std_df


,Source Error Std,Target Error Std
D4-50MvitC0.0WD0.2LR0.4M0.1,23.749646,26.582628
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293
D4-50MvitC0.2WD0.2LR0.4M0.1,23.749646,25.645169
D4-50MvitC0.3WD0.2LR0.4M0.1,23.749646,25.346264
D4-50MvitC0.9WD0.2LR0.4M0.1,23.749646,25.011493


In [19]:
std_df["Error Std Percentual Change"] = (
    100
    * (std_df["Target Error Std"] - std_df["Source Error Std"])
    / std_df["Source Error Std"]
)
# std_df["Error Std Absolute Change"] = std_df["Target Error Std"] - std_df["Source Error Std"]
std_df_change = std_df[["Error Std Percentual Change"]]
std_df


,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.0WD0.2LR0.4M0.1,23.749646,26.582628,11.928521
D4-50MvitC0.1WD0.2LR0.4M0.1,23.749646,26.057293,9.716553
D4-50MvitC0.2WD0.2LR0.4M0.1,23.749646,25.645169,7.981267
D4-50MvitC0.3WD0.2LR0.4M0.1,23.749646,25.346264,6.722703
D4-50MvitC0.9WD0.2LR0.4M0.1,23.749646,25.011493,5.313116


In [20]:
mean_df.join(std_df)


,Source Error Mean,Target Error Mean,Error Mean Percentual Change,Source Error Std,Target Error Std,Error Std Percentual Change
D4-50MvitC0.0WD0.2LR0.4M0.1,59.351015,52.256427,-11.953609,23.749646,26.582628,11.928521
D4-50MvitC0.1WD0.2LR0.4M0.1,59.351015,53.890648,-9.200124,23.749646,26.057293,9.716553
D4-50MvitC0.2WD0.2LR0.4M0.1,59.351015,55.143210,-7.089694,23.749646,25.645169,7.981267
D4-50MvitC0.3WD0.2LR0.4M0.1,59.351015,55.883476,-5.842426,23.749646,25.346264,6.722703
D4-50MvitC0.9WD0.2LR0.4M0.1,59.351015,56.938571,-4.064706,23.749646,25.011493,5.313116


In [22]:
all_df = params_df[["confidence"]].join(mean_df).join(std_df)
all_df.to_latex(float_format="%.2f")


/tmp/ipykernel_90/828120639.py:2: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  all_df.to_latex(float_format="%.2f")


'\\begin{tabular}{lrrrrrrr}\n\\toprule\n{} &  confidence &  Source Error Mean &  Target Error Mean &  Error Mean Percentual Change &  Source Error Std &  Target Error Std &  Error Std Percentual Change \\\\\n\\midrule\nD4-50MvitC0.0WD0.2LR0.4M0.1 &        0.00 &              59.35 &              52.26 &                        -11.95 &             23.75 &             26.58 &                        11.93 \\\\\nD4-50MvitC0.1WD0.2LR0.4M0.1 &        0.10 &              59.35 &              53.89 &                         -9.20 &             23.75 &             26.06 &                         9.72 \\\\\nD4-50MvitC0.2WD0.2LR0.4M0.1 &        0.20 &              59.35 &              55.14 &                         -7.09 &             23.75 &             25.65 &                         7.98 \\\\\nD4-50MvitC0.3WD0.2LR0.4M0.1 &        0.30 &              59.35 &              55.88 &                         -5.84 &             23.75 &             25.35 &                         6.72 \\\\\nD4-50Mvit

In [36]:
all_df_base = params_df[["model", "dataset"]].join(mean_df_base).join(std_df_base)
all_df_base


,model,dataset,Source Error Mean,Target Error Mean,Source Error Std,Target Error Std
D4-50MvitC0.1WD0.2LR0.4M0.1,vit,4-50,59.351015,53.890648,23.749646,26.057293
D4-50MvitC0.2WD0.2LR0.4M0.1,vit,4-50,59.351015,55.143210,23.749646,25.645169
D4-50MvitC0.3WD0.2LR0.4M0.1,vit,4-50,59.351015,55.883476,23.749646,25.346264
D4-50MvitC0.9WD0.2LR0.4M0.1,vit,4-50,59.351015,56.938571,23.749646,25.011493


In [37]:
all_df_change = params_df[["model", "dataset"]].join(mean_df_change).join(std_df_change)
all_df_change


,model,dataset,Error Mean Percentual Change,Error Std Percentual Change
D4-50MvitC0.1WD0.2LR0.4M0.1,vit,4-50,-9.200124,9.716553
D4-50MvitC0.2WD0.2LR0.4M0.1,vit,4-50,-7.089694,7.981267
D4-50MvitC0.3WD0.2LR0.4M0.1,vit,4-50,-5.842426,6.722703
D4-50MvitC0.9WD0.2LR0.4M0.1,vit,4-50,-4.064706,5.313116


In [26]:
def find_worse(better_id: str, worse_id: str):
    runs = client.search_runs(experiment_ids=[better_id, worse_id], max_results=2000)
    ekey = lambda run: run.info.experiment_id
    runs.sort(key=ekey)

    experiment_runs = dict()
    for k, group in itertools.groupby(runs, key=ekey):
        experiment_runs[k] = {run.data.params["Filename"]: run for run in group}

    metric = "Target Accumulated Top-1 Count"
    worsened = []
    for filename, run in experiment_runs[worse_id].items():
        if (
            experiment_runs[better_id][filename].data.metrics[metric]
            < run.data.metrics[metric]
        ):
            worsened.append(filename)
    
    rows = []
    for filename in worsened:
        baseline = "Source Accumulated Top-1 Count"
        metric = "Target Accumulated Top-1 Count"

        row = {
            "Filename": filename,
            baseline: experiment_runs[worse_id][filename].data.metrics[baseline],
            f"{worse_id}-{metric}": experiment_runs[worse_id][filename].data.metrics[metric],
            f"{better_id}-{metric}": experiment_runs[better_id][filename].data.metrics[metric],
        }
        rows.append(row)

    return pd.DataFrame(rows)
            

In [27]:
experiments

[<Experiment: artifact_location='mlflow-artifacts:/52', creation_time=1669509831923, experiment_id='52', last_update_time=1669509831923, lifecycle_stage='active', name='D4-50MvitC0.0WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/32', creation_time=1669040748038, experiment_id='32', last_update_time=1669040748038, lifecycle_stage='active', name='D4-50MvitC0.1WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/38', creation_time=1669120072652, experiment_id='38', last_update_time=1669120072652, lifecycle_stage='active', name='D4-50MvitC0.2WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/39', creation_time=1669120072692, experiment_id='39', last_update_time=1669120072692, lifecycle_stage='active', name='D4-50MvitC0.3WD0.2LR0.4M0.1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/40', creation_time=1669120072714, experiment_id='40', last_update_time=1669120072714, lifecycle_stage='active', n

In [29]:
better_id = "32"  # 0.0
worse_id = "39"  # 0.1


In [30]:
df = find_worse(better_id=better_id, worse_id=worse_id)
df

,Filename,Source Accumulated Top-1 Count,39-Target Accumulated Top-1 Count,32-Target Accumulated Top-1 Count
0,/datasets/imagenet1k-4-50/975-lakeside-4-40.mp4,6.0,9.0,7.0
1,/datasets/imagenet1k-4-50/887-vestment-4-40.mp4,5.0,3.0,2.0
2,/datasets/imagenet1k-4-50/871-trimaran-4-40.mp4,26.0,24.0,20.0
3,/datasets/imagenet1k-4-50/870-tricycle-4-40.mp4,6.0,6.0,5.0
4,/datasets/imagenet1k-4-50/751-racer-4-40.mp4,3.0,2.0,0.0
5,/datasets/imagenet1k-4-50/74-garden_spider-4-4...,4.0,2.0,1.0
6,/datasets/imagenet1k-4-50/703-park_bench-4-40.mp4,7.0,6.0,5.0
7,/datasets/imagenet1k-4-50/665-moped-4-40.mp4,18.0,17.0,16.0
8,/datasets/imagenet1k-4-50/655-miniskirt-4-40.mp4,15.0,16.0,15.0
9,/datasets/imagenet1k-4-50/621-lawn_mower-4-40.mp4,9.0,16.0,14.0
